In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy import spatial
from tqdm.notebook import tqdm

from src.normals import estim_normals_spline
from src.orientation import orient_normals_cvx
from src.plotting import set_axes_equal, set_defense_context
from src.utils import remove_hidden_points, edblquad, estimate_surface_area

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# constants

A = 4  # averaging area in cm2
a = np.sqrt(A)  # a side of the square averaging surface
rc = a / np.sqrt(2)  # radius of a circumscribed circle of the averaging surface

# Data

In [ ]:
def sphere(r=5, samples=1000):
    # https://stackoverflow.com/a/44164075/15005103
    i = np.arange(0, samples, dtype=float) + 0.5
    phi = np.arccos(1 - 2 * i / samples)
    theta = np.pi * (1 + np.sqrt(5)) * i
    x = r * np.cos(theta) * np.sin(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(phi)
    return np.c_[x, y, z]

In [ ]:
# generate coordinates of a sphere

r = 5
samples = 50_000
xyz = sphere(r, samples)
n = estim_normals_spline(xyz, k=30, unit=False)
n = orient_normals_cvx(xyz, n)

In [ ]:
# create 2 synthetic absorbed power density patterns

mask = np.where(xyz[:, 0] > 0)[0]  # the wave propagates in the x-direction
y = xyz[mask, 1]
z = xyz[mask, 2]

center_1 = [0, 0]
radius_1 = 0.5
region_1 = np.sqrt(((y - center_1[0]) / 1.5) ** 2 + (z - center_1[1]) ** 2)
apd_1 = 15.1 * np.exp(-(region_1 / radius_1) ** 2)

center_2 = [2.5, -1]
radius_2 = 1.5
region_2 = np.sqrt((y - center_2[0]) ** 2 + (z - center_2[1]) ** 2)
apd_2 = 10 * np.exp(-(region_2 / radius_2) ** 2)

apd = np.zeros((xyz.shape[0], ))
apd[mask] = apd_1 + apd_2

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, marker='D', s=1, c=apd)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.quiver(9, 0, 0, -2.5, 0, 0, lw=1, color='w')
    ax.text(9, 0, 0.5, s='$\\vec k$', color='w', zdir='x')
    ax.set(xlabel='$x$ [cm]', ylabel='$y$ [cm]', zlabel='$z$ [cm]')
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(10, 30)
    plt.show()
    fname = 'exposure-scenario.png'
    fig.savefig(os.path.join('figures', 'hotspot-detection', fname),
                dpi=500, bbox_inches='tight')

# Hidden point removal

This allows for less computational demand as we will investigate regions that are directly irradiated and visible from the point of view of the EM field incidence.

In [ ]:
# remove points not visible from specified point of view - reverse x-direction

center = np.mean(xyz, axis=0)
diameter = np.linalg.norm(xyz.max(axis=0) - xyz.min(axis=0))
pov = center.copy()
pov[0] += diameter
vis = remove_hidden_points(xyz, pov, np.pi)
xyz_vis = xyz[vis]
apd_vis = apd[vis]

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz_vis.T, c=apd_vis, marker='D', s=1)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.quiver(9, 0, 0, -2.5, 0, 0, lw=1, color='k')
    ax.text(8, 0, 0.5, s='$\\vec k$', color='k', zdir='x')
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    ax.view_init(0, 90)
    plt.show()
    fname = 'exposure-scenario-hpr.png'
    fig.savefig(os.path.join('figures', 'hotspot-detection', fname),
                dpi=500, bbox_inches='tight')

# Fixed POV

Automatic hot-spot area detection (worst case exposure scenario) by iterating through all points where the projection of the averaging surface is placed perpendicular to the k-vector and mapped onto the non-planar exposed surface.

In [ ]:
res_1 = {'p': [],
         'nbh': [],
         'nbh_ind': [],
         'area': [],
         'apd': [],
         'sapd': []}

In [ ]:
res_1.fromkeys(res_1, [])

for p in tqdm(xyz_vis):
    # bounding box around each point perpendicular to the k vector (yz-plane)
    _bbox = [p[1]-a/2, p[1]+a/2,
             p[2]-a/2, p[2]+a/2]
    _bbox_ind = np.where((xyz_vis[:, 1] >= _bbox[0])
                         & (xyz_vis[:, 1] <= _bbox[1])
                         & (xyz_vis[:, 2] >= _bbox[2])
                         & (xyz_vis[:, 2] <= _bbox[3]))[0]
    _nbh = xyz_vis[_bbox_ind]
    _apd = apd_vis[_bbox_ind]
    
    # create a convex hull for the local point cloud within the bounding box
    _hull = spatial.ConvexHull(_nbh[:, 1:])
    
    # compute the area of a convex hull by considering yz-plane
    _hull_area = _hull.volume  # for 2-D `volume` results with an area
    
    # relative % difference between control area and the area of a convex hull
    rpd = np.abs(_hull_area - A) / A
    if rpd > 0.1:
        continue  # skip iteration if the difference is too large
    else:
        # fit a local surface
        fi = interpolate.SmoothBivariateSpline(*_nbh[:, 1:].T,  # yz
                                               _nbh[:, 0],      # x
                                               bbox=_bbox,
                                               s=0.1)
        # compute the normals on a local surface
        _n = np.c_[fi(*_nbh[:, 1:].T, dx=1, grid=False),   # dy
                   fi(*_nbh[:, 1:].T, dy=1, grid=False),   # dz
                   np.ones((_nbh.shape[0]))]
        _n_mag = np.linalg.norm(_n, axis=1)
        _area = edblquad(_nbh[:, 1:], _n_mag, bbox=_bbox)
        _sapd = 1 / _area * edblquad(_nbh[:, 1:], _apd, bbox=_bbox)
        
        # capture the results
        res_1['p'].append(p)
        res_1['nbh'].append(_nbh)
        res_1['nbh_ind'].append(_bbox_ind)
        res_1['area'].append(_area)
        res_1['apd'].append(_apd)
        res_1['sapd'].append(_sapd)
df_res_1 = pd.DataFrame(res_1)

In [ ]:
# extract the spatial maximum absorbed power density

p_p_1, p_nbh_1, p_nbh_ind_1, p_area_1, p_apd_1, p_sapd_1 = df_res_1[
    df_res_1['sapd'] == df_res_1['sapd'].max()
].to_numpy()[0].T

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*p_nbh_1.T, c=p_apd_1, s=10)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           title=(r'$ps\text{PD}_{n}$ = '
                  f'{p_sapd_1:.2f} W/m$^2$'),
           xticks=[], xticklabels=[])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    plt.show()
    fname = 'apd-fixed-dist.png'
    fig.savefig(os.path.join('figures', 'hotspot-detection', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*np.delete(xyz_vis, p_nbh_ind_1, axis=0).T,
               marker='D', fc='w', ec='k', s=0.25, alpha=0.25)
    ax.scatter(*p_nbh_1.T,
               marker='D', c=p_apd_1, s=0.25)
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           xticks=[], xticklabels=[],
           yticks=[-r, 0, r],
           zticks=[-r, 0, r])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    plt.show()
    fname = 'apd-fixed-pos.png'
    fig.savefig(os.path.join('figures', 'hotspot-detection', fname),
                dpi=500, bbox_inches='tight')

# POV perpendicular to the normal direction

Automatic hot spot area detection by iterating through all points where the averaging surface is placed perpendicular to the normal vector at the current center point and mapped to the non-planar exposed surface with regards to an orthonormal basis.

In [ ]:
res_2 = {'p': [],
         'nbh': [],
         'area': [],
         'apd': [],
         'sapd': []}

In [ ]:
res_2.fromkeys(res_2, [])  # reset values of the dictionary

tree = spatial.KDTree(xyz)  # k-D tree of the point cloud
for p in tqdm(xyz_vis):  # run only for the "visible" set of points
    # find all points within radius rc, i.e., from rc/(1+eps) to rc*(1+eps)
    _nbh_ind = tree.query_ball_point([p], rc, eps=0.01, workers=-1)[0]
    _nbh = xyz[_nbh_ind]
    _n = n[_nbh_ind]
    _n_mag = np.linalg.norm(_n, axis=1)
    _apd = apd[_nbh_ind]
    
    # map a local point cloud to an orthonormal basis
    X = _nbh.copy()
    mu = X.mean(axis=0)
    Xn = X - mu
    pn = p - mu
    C = Xn.T @ Xn  # covariance matrix
    U, _, _ = np.linalg.svd(C)
    Xt = Xn @ U
    pt = pn @ U
    
    # create two parametrized variables that lie on a tangent plane
    u, v = Xt[:, :2].T
    pu, pv = pt[:2]
    
    # set a bounding box that corresponds to the control square area
    _bbox = [pu-a/2, pu+a/2,
             pv-a/2, pv+a/2]
    _bbox_ind = np.where((u >= _bbox[0])
                         & (u <= _bbox[1])
                         & (v >= _bbox[2])
                         & (v <= _bbox[3]))[0]
    
    # estimate the averaging area
    _area = edblquad(points=Xt[_bbox_ind, :2],
                     values=_n_mag[_bbox_ind],
                     bbox=_bbox,
                     s=0.1)
    
    # estimate the spatially-averaged absorbed power density
    _sapd = 1 / _area * edblquad(points=Xt[_bbox_ind, :2],
                                 values=_apd[_bbox_ind],
                                 bbox=_bbox,
                                 s=1)
    
    # capture the results
    res_2['p'].append(p)
    res_2['nbh'].append(_nbh[_bbox_ind])
    res_2['area'].append(_area)
    res_2['apd'].append(_apd[_bbox_ind])
    res_2['sapd'].append(_sapd)
df_res_2 = pd.DataFrame(res_2)

In [ ]:
# extract the spatial maximum absorbed power density

p_p_2, p_nbh_2, p_area_2, p_apd_2, p_sapd_2 = df_res_2[
    df_res_2['sapd'] == df_res_2['sapd'].max()
].to_numpy()[0].T

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*p_nbh_2.T, c=p_apd_2, s=10)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           title=(r'$ps\text{PD}_{n}$ = '
                  f'{p_sapd_2:.2f} W/m$^2$'),
           xticks=[], xticklabels=[])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    plt.show()
    fname = 'apd-adapt-dist.png'
    fig.savefig(os.path.join('figures', 'hotspot-detection', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
_, mask = np.where((xyz_vis == p_nbh_2[:, np.newaxis]).sum(axis=2)==3)
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*np.delete(xyz_vis, mask, axis=0).T,
               marker='D', fc='w', ec='k', s=0.25, alpha=0.25)
    ax.scatter(*p_nbh_2.T,
               marker='D', c=p_apd_2, s=0.25)
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           xticks=[], xticklabels=[],
           yticks=[-r, 0, r],
           zticks=[-r, 0, r])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    plt.show()
    fname = 'apd-adapt-pos.png'
    fig.savefig(os.path.join('figures', 'hotspot-detection', fname),
                dpi=500, bbox_inches='tight')

# Animation

In [ ]:
from IPython import display
import time

In [ ]:
xyz_sample = df_res_2.nlargest(500, columns='sapd')['p'].values[::-5]
tree = spatial.KDTree(xyz)
for i, p in enumerate(tqdm(xyz_sample)):
    _nbh_ind = tree.query_ball_point([p], rc, eps=0.01, workers=-1)[0]
    _nbh = xyz[_nbh_ind]
    _n = n[_nbh_ind]
    _n_mag = np.linalg.norm(_n, axis=1)
    _apd = apd[_nbh_ind]
    
    X = _nbh.copy()
    mu = X.mean(axis=0)
    Xn = X - mu
    pn = p - mu
    C = Xn.T @ Xn
    U, _, _ = np.linalg.svd(C)
    Xt = Xn @ U
    pt = pn @ U
    
    u, v = Xt[:, :2].T
    pu, pv = pt[:2]
    _bbox = [pu-a/2, pu+a/2,
             pv-a/2, pv+a/2]
    _bbox_ind = np.where((u >= _bbox[0])
                         & (u <= _bbox[1])
                         & (v >= _bbox[2])
                         & (v <= _bbox[3]))[0]
    
    _area = edblquad(points=Xt[_bbox_ind, :2],
                     values=_n_mag[_bbox_ind],
                     bbox=_bbox,
                     s=0.1)
    
    _sapd = 1 / _area * edblquad(points=Xt[_bbox_ind, :2],
                                 values=_apd[_bbox_ind],
                                 bbox=_bbox,
                                 s=1)
    
    # visualize
    _, mask = np.where((xyz_vis == _nbh[_bbox_ind, np.newaxis]).sum(axis=2)==3)
    with set_defense_context():
        plt.clf()
        fig = plt.figure(figsize=(9, 4))

        # left subfigure
        ax1 = fig.add_subplot(1, 2, 1, projection ='3d')
        ax1.scatter(*np.delete(xyz_vis, mask, axis=0).T,
                    marker='D', fc='w', ec='k', s=0.2, alpha=0.2)
        ax1.scatter(*_nbh[_bbox_ind].T, c=_apd[_bbox_ind],
                    marker='o', s=1)
        ax1.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
                xticks=[], xticklabels=[],
                yticks=[-r, 0, r],
                zticks=[-r, 0, r])
        ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax1.set_box_aspect([1, 1, 1])
        ax1 = set_axes_equal(ax1)
        ax1.view_init(0, 0)

        # right subfigure
        ax2 = fig.add_subplot(1, 2, 2)
        s = ax2.scatter(*Xt[_bbox_ind, :2].T, c=_apd[_bbox_ind])
        cbar = fig.colorbar(s, ax=ax2, format='%.0e',
                            label='power density [W/m$^2$]')
        ax2.set(xlabel='$u$', ylabel='$v$',
                xticks=[-1, 0, 1], xticklabels=[-1, 0, 1],
                xlim=[-1.1, 1.1],
                yticks=[-1, 0, 1], yticklabels=[-1, 0, 1],
                ylim=[-1.1, 1.1],
                title=(r'$ps\text{PD}_{n}$ = '
                       f'{_sapd:.2f} W/m$^2$'),)
        ax2.set_aspect('equal', 'box')

        plt.tight_layout()
        plt.show()
        display.clear_output(wait=True)
        fname = f'hs-anim-{i:02d}.png'
        fig.savefig(os.path.join('figures',
                                 'hotspot-detection',
                                 'animation',
                                 fname),
                    dpi=250, bbox_inches='tight')

In [ ]:
_, mask = np.where((xyz_vis == _nbh[_bbox_ind, np.newaxis]).sum(axis=2)==3)
with set_defense_context():
    fig = plt.figure(figsize=(9, 4))

    # left subfigure
    ax1 = fig.add_subplot(1, 2, 1, projection ='3d')
    ax1.scatter(*np.delete(xyz_vis, mask, axis=0).T,
                marker='D', fc='w', ec='k', s=0.2, alpha=0.2)
    ax1.scatter(*_nbh[_bbox_ind].T, c=_apd[_bbox_ind],
                marker='o', s=1)
    ax1.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
            xticks=[], xticklabels=[],
            yticks=[-r, 0, r],
            zticks=[-r, 0, r])
    ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax1.set_box_aspect([1, 1, 1])
    ax1 = set_axes_equal(ax1)
    ax1.view_init(0, 0)

    # right subfigure
    ax2 = fig.add_subplot(1, 2, 2)
    s = ax2.scatter(*Xt[_bbox_ind, :2].T, c=_apd[_bbox_ind])
    cbar = fig.colorbar(s, ax=ax2, format='%.0e',
                        label='power density [W/m$^2$]')
    ax2.set(xlabel='$u$', ylabel='$v$',
            xticks=[-1, 0, 1], xticklabels=[-1, 0, 1],
            xlim=[-1.1, 1.1],
            yticks=[-1, 0, 1], yticklabels=[-1, 0, 1],
            ylim=[-1.1, 1.1],
            title=(r'$ps\text{PD}_{n}$ = '
                   f'{_sapd:.2f} W/m$^2$'),)
    ax2.set_aspect('equal', 'box')

    plt.tight_layout()
    plt.show()
    fname = 'apd-adapt-anim.pdf'
    fig.savefig(os.path.join('figures',
                             'hotspot-detection',
                             fname),
                bbox_inches='tight')